In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import openpyxl

In [2]:
##DO Cleaning
df = pd.read_csv('DataAnalyst_Ecom_data_sessionCounts.csv')
df['dim_date'] = df['dim_date'].str.replace('/','-')
df[['Month','Day','Year']] = df['dim_date'].str.split('-', expand=True)

cols = ['Month','Day','Year']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
df.loc[df['Year'] < 2000, 'Year'] = df['Year']+ 2000

df['Month'] = df['Month'].apply(lambda x: '{0:0>2}'.format(x))
df['Day'] = df['Day'].apply(lambda x: '{0:0>2}'.format(x))
df['Date'] = df['Day'].map(str)+'/'+df['Month'].map(str)+'/'+df['Year'].map(str)
df['Date']=pd.to_datetime(df['Date'],format='mixed',dayfirst=True)

df_copy=df.copy()
df_copy['key']=df_copy['Month'].map(str)+'-'+df_copy['Year'].map(str)

In [3]:
# #sheet 1
df_copy1=df_copy[['key','dim_deviceCategory','sessions','transactions', 'QTY']]
df_clean=df_copy1.groupby(['key','dim_deviceCategory']).sum().reset_index()
df_clean['ECR'] = df_clean['transactions']/df_clean['sessions'].replace(np.inf, 0).round()

In [4]:
#sheet 2
df2=df_copy[['Month','Year','key','sessions','transactions','QTY']].apply(pd.to_numeric,errors='ignore')
df3=df2.groupby(['Month','Year','key']).sum().reset_index()
df3['ECR'] = df3['transactions']/df3['sessions'].replace(np.inf, 0).round()
df_add_cart = pd.read_csv('DataAnalyst_Ecom_data_addsToCart.csv')
df_add_cart['dim_month'] = df_add_cart['dim_month'].apply(lambda x: '{0:0>2}'.format(x))
df_add_cart['key_c']=df_add_cart['dim_month'].map(str)+'-'+df_add_cart['dim_year'].map(str)
df_merge=pd.merge(df3,df_add_cart,left_on='key',right_on='key_c')
df5=df_merge[['key_c','Month','Year','sessions','transactions','QTY','ECR','addsToCart']]
df5['Value']=df5['transactions']*df5['QTY']
df5['key_c'] = pd.to_datetime(df5['key_c'],format='%m-%Y')
df5.sort_values(by='key_c',ascending=True,inplace=True)
df5['%-change-Value']=df5['Value'].pct_change()

C:\Users\Shivangi\AppData\Local\Temp\ipykernel_12704\3725308299.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Value']=df5['transactions']*df5['QTY']
C:\Users\Shivangi\AppData\Local\Temp\ipykernel_12704\3725308299.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['key_c'] = pd.to_datetime(df5['key_c'],format='%m-%Y')
C:\Users\Shivangi\AppData\Local\Temp\ipykernel_12704\3725308299.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

In [5]:
#Extra analyssis
df_browser=df_copy.groupby('dim_browser')[['sessions','transactions','QTY']].agg('sum')
df_brow=pd.DataFrame(df_browser)
df_brow=df_brow.sort_values('sessions',ascending=False).reset_index()

In [6]:
#Write all above sheets into a Excel Workbook
with pd.ExcelWriter('Assignment.xlsx') as writer:
    df_clean.to_excel(writer, sheet_name='Month-x-Device',index=False)
    df5.to_excel(writer, sheet_name='Month-on-Month-Comparison',index=False)
    df_browser.to_excel(writer, sheet_name='BrowserVsKPI',index=True)